In [238]:
import quandl
import pandas as pd
import math
import numpy as np
from sklearn import preprocessing, svm, model_selection
from sklearn.linear_model import LinearRegression


In [239]:
df = quandl.get("EOD/HD", authtoken="YDkkGsZnZYV4jyxNANZ1")

In [240]:
print(df.head())

             Open   High    Low  Close      Volume  Dividend  Split  \
Date                                                                  
2013-09-03  74.83  75.25  73.68  73.90   7802800.0      0.39    1.0   
2013-09-04  73.98  74.51  73.74  74.14   8271600.0      0.00    1.0   
2013-09-05  74.07  74.21  72.84  72.99  11039000.0      0.00    1.0   
2013-09-06  73.55  73.56  72.21  72.70   9968400.0      0.00    1.0   
2013-09-09  72.98  73.79  72.90  73.58   5781800.0      0.00    1.0   

             Adj_Open   Adj_High    Adj_Low  Adj_Close  Adj_Volume  
Date                                                                
2013-09-03  62.854838  63.207625  61.888874  62.073667   7802800.0  
2013-09-04  62.140865  62.586048  61.939272  62.275260   8271600.0  
2013-09-05  62.216462  62.334058  61.183301  61.309296  11039000.0  
2013-09-06  61.779678  61.788078  60.654121  61.065705   9968400.0  
2013-09-09  61.300896  61.981271  61.233699  61.804878   5781800.0  


In [241]:
df = df[['Adj_Open','Adj_High','Adj_Low','Adj_Close','Adj_Volume']]

In [242]:
df.head()

,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,
2013-09-03,62.854838,63.207625,61.888874,62.073667,7802800.0
2013-09-04,62.140865,62.586048,61.939272,62.275260,8271600.0
2013-09-05,62.216462,62.334058,61.183301,61.309296,11039000.0
2013-09-06,61.779678,61.788078,60.654121,61.065705,9968400.0
2013-09-09,61.300896,61.981271,61.233699,61.804878,5781800.0


In [243]:
df['HL_PCT'] = ((df['Adj_High']-df['Adj_Close'])/df['Adj_Close'])*100
df['DAY_PCT'] = ((df['Adj_Close']-df['Adj_Open'])/df['Adj_Open'])*100

In [244]:
df.head()

,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,HL_PCT,DAY_PCT
Date,,,,,,,
2013-09-03,62.854838,63.207625,61.888874,62.073667,7802800.0,1.826793,-1.242817
2013-09-04,62.140865,62.586048,61.939272,62.275260,8271600.0,0.499056,0.216275
2013-09-05,62.216462,62.334058,61.183301,61.309296,11039000.0,1.671462,-1.458080
2013-09-06,61.779678,61.788078,60.654121,61.065705,9968400.0,1.182944,-1.155676
2013-09-09,61.300896,61.981271,61.233699,61.804878,5781800.0,0.285404,0.822143


In [245]:
df =df[['Adj_Close','HL_PCT','DAY_PCT','Adj_Volume']] # Get rid of alluseless data
df.head()

,Adj_Close,HL_PCT,DAY_PCT,Adj_Volume
Date,,,,
2013-09-03,62.073667,1.826793,-1.242817,7802800.0
2013-09-04,62.275260,0.499056,0.216275,8271600.0
2013-09-05,61.309296,1.671462,-1.458080,11039000.0
2013-09-06,61.065705,1.182944,-1.155676,9968400.0
2013-09-09,61.804878,0.285404,0.822143,5781800.0


In [246]:
predict_offset = int(math.ceil(0.01*len(df))) # this number decides how far into future are we pridicting
prediction_col = 'Adj_Close'
print(predict_offset)

11


In [247]:
df['label'] = df[prediction_col].shift(-predict_offset)

In [248]:
df.head #so here we ar basically fetching close prices from 11 days into future and defining it as lable for today and we will be predicting close price of a day 11 days future as well

<bound method NDFrame.head of              Adj_Close    HL_PCT   DAY_PCT  Adj_Volume      label
Date                                                             
2013-09-03   62.073667  1.826793 -1.242817   7802800.0  64.988358
2013-09-04   62.275260  0.499056  0.216275   8271600.0  65.945922
2013-09-05   61.309296  1.671462 -1.458080  11039000.0  64.677570
2013-09-06   61.065705  1.182944 -1.155676   9968400.0  63.762004
2013-09-09   61.804878  0.285404  0.822143   5781800.0  63.871200
...                ...       ...       ...         ...        ...
2017-12-21  172.924169  0.404083  0.202451   5859058.0        NaN
2017-12-22  172.970140  0.175411 -0.037194   3256519.0        NaN
2017-12-26  175.020443  0.031519  0.970668   2969182.0        NaN
2017-12-27  174.864142  0.683527 -0.215110   5912613.0        NaN
2017-12-28  174.487180  0.632311 -0.591902   3175631.0        NaN

[1090 rows x 5 columns]>

since lables from now are shifter into 11 days into past there are some NaN lablesin present

In [249]:
df.dropna(inplace = True)

In [250]:
df.head()

,Adj_Close,HL_PCT,DAY_PCT,Adj_Volume,label
Date,,,,,
2013-09-03,62.073667,1.826793,-1.242817,7802800.0,64.988358
2013-09-04,62.275260,0.499056,0.216275,8271600.0,65.945922
2013-09-05,61.309296,1.671462,-1.458080,11039000.0,64.677570
2013-09-06,61.065705,1.182944,-1.155676,9968400.0,63.762004
2013-09-09,61.804878,0.285404,0.822143,5781800.0,63.871200


In [251]:
X = preprocessing.scale(np.array(df.drop(['label'],1)))# scaling data so that the data is in range(-1,1) helps in accuracy and training time
y = np.array(df['label'])

In [252]:
X

array([[-1.5559049 ,  1.78372482, -1.36613676,  1.0445421 ],
       [-1.54836059, -0.31123899,  0.19015735,  1.25021977],
       [-1.58451043,  1.53863634, -1.59574039,  2.46436737],
       ...,
       [ 2.43183549, -0.67713317,  0.4913229 , -0.14497596],
       [ 2.39192228, -0.4839832 , -0.41958587,  0.27065835],
       [ 2.3764387 ,  0.12506951, -0.59499168, -0.06451378]])

In [253]:
X_train , X_test , y_train , y_test = model_selection.train_test_split(X , y , test_size = 0.2)

In [256]:
clf = LinearRegression (n_jobs = -1) #n_jobs is an attribute we can decide how many threads are used for the training -1 means as many as possible
clf.fit(X_train , y_train)
accuracy = clf.score(X_test,y_test)
accuracy #this is basically squared error

0.9854600641218598

# instead of using linear trying svm

In [237]:
clf = svm.SVR()
clf.fit(X_train , y_train)
accuracy = clf.score(X_test,y_test)
accuracy

0.9387934763585375